# Homework 3 - Find the perfect place to stay in Texas!

###### Alessandro Flaborea, Egon Ferri, Melis Kaymaz

The homework consists in analyzing the text of Airbnb property listings and building a search engine.

In [1]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk import pos_tag, ne_chunk
import nltk
import math
import json
from geopy import distance
import numpy
import heapq_max
import folium
import os
import branca.colormap as cm
import  requests
from  bs4  import  BeautifulSoup
import re
import csv
from tkinter import *
from functions import *

## Step 2: Create documents

The first task is to create a `.tsv` file for each record of the dataset.
First thing to do is to read the `.csv` file.

In [3]:
# opening the csv file containing all the rooms in Texas
f = pd.read_csv(r'C:\Users\Egon\Desktop\Universita\ADM\HW3\Airbnb_Texas_Rentals.csv')


# taking only the average_rate_per_night for each entry in the dataframe
f['price_rank'] = list(map(lambda x: str(x)[1:], f['average_rate_per_night']))


Now we can create `.tsv` files and store them in a folder.

In [ ]:
# creating all the tsv files from the dataframe
for i in range(f.index.max()+1):
    op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\doc\doc_' + str(i) + '.tsv', 'w', encoding="utf-8")
    for j in range(10):
        op.write('%s\t' %f.iloc[i, j])
    op.close()

## Step 3: Search Engine

Now, we want to create two different Search Engines that, given as input a query, return the houses that match the query.

As a first common step, we want to preprocess the documents by

1. Removing stopwords
2. Removing punctuation
3. Stemming

Then we want to build a file named `vocabulary.txt`, that maps each word to an integer.

In [71]:
# creating the vocabulary that maps a word to an integer

vocabulary= {}
index = 0

for i in range(18259):
    
    op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\doc\doc_' + str(i) + '.tsv', 'r', encoding="utf-8")
    for line in op:
        ou = line.strip().split('\t')
        sentence = ou[5].replace('\\n', ' ').replace('/', ' ').replace('*', ' ').replace('\\r', ' ').replace('\\t', ' ') + ' ' + ou[8].replace('\\n', ' ').replace('/', ' ').replace('*', ' ').replace('\\r', ' ').replace('\\t', ' ')
    op.close()
        
    #preprocessing data deleting stop words, punctuations, ecc.  
    final = preprocess(word_tokenize(sentence))
    
    # IF  word not in vocabulary -> add the word
    vocabulary, index = vocabularization(vocabulary, final, index)
            
op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\vocabulary.txt', 'w', encoding="utf-8")
op.write(json.dumps(vocabulary))
op.close()

In [4]:
# opening the created vocabulary.txt
op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\vocabulary.txt', 'r', encoding="utf-8")
vocabulary = json.loads(op.read())

### 3.1) Conjunctive query
At this moment, we narrow out interest on the `description` and `title` of each document. It means that the first Search Engine will evaluate queries with respect to the aforementioned information.

#### 3.1.1) Creating our index!

We want to create the Inverted Index. It will be a dictionary of this format:

```
{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}
```

where _document\_i_ is the *id* of a document that contains the word.

We also want to store it in a separate file, `inverted_index.txt` , and load it in memory with json when needed.

In [ ]:
# from the vocabulary, tha inverted index is created
inverted_index = {}

for file in range(18259):

    op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\doc\doc_' + str(file) + '.tsv', 'r', encoding="utf-8")
    for line in op:
        ou = line.strip().split('\t')
        sentence = ou[5].replace('\\n', ' ').replace('/', ' ').replace('*', ' ').replace('\\r', ' ').replace('\\t', ' ') + ' ' + ou[8].replace('\\n', ' ').replace('/', ' ').replace('*', ' ').replace('\\r', ' ').replace('\\t', ' ')
    op.close()
 
    
    # preprocessing data deleting stop words, punctuations, ecc.  
    final = preprocess(word_tokenize(sentence))
    
    
    # CREATING INVERTED INDEX
    for word in final:
        index = vocabulary[word]
        if not (index in inverted_index):
            inverted_index[index] = ['doc_' + str(file)]
        elif not('doc_' + str(file) in inverted_index[index]):
            inverted_index[index] = inverted_index[index] + ['doc_' + str(file)]

op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\inverted_index.txt', 'w', encoding="utf-8")
op.write(json.dumps(inverted_index))
op.close()

In [5]:
# opening the created inverted_index.txt

op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\inverted_index.txt', 'r', encoding="utf-8")
inverted_index = json.loads(op.read())


#### 3.1.2) Execute the query
Now given a query, that we let the user enter, for example:
```
a queen and comfortable room with netflix
```
we want that the Search Engine returns a list of documents that contains all the important words in the query.

Handling the query:

In [4]:
# handling the query
user_query = input('What are you searching?\n')
    
# preprocessing data deleting stop words, punctuations, ecc.  
# final contains the processed words
query = preprocess(word_tokenize(user_query))


What are you searching?
a queen and comfortable room with netflix


Creating an inverted index containing only the words of the query:

In [38]:
# taking from the inverted index only the words present in the query
voc = {}
inverted_index_query = {}
i=0
for word in query: 
    voc[i]  = vocabulary[word]
    i = i+1
for index in range(i):
    inverted_index_query[voc[index]] = inverted_index[voc[index]]


Finding list of docs that contain all the words in the query and printing them in the format that we want:

In [39]:
# finding list of docs that contains all the words in the query
docs = []

for i in range(18259):
    doc = 'doc_' + str(i)
    b = True
    for j in voc.values(): 
        b = b and (doc in inverted_index_query[j])
    if b:
        docs.append(i)

# printing the table with docs with all the words in the query
df = f.filter(items = ['title', 'description', 'city', 'url']).loc[docs]
df.description = list(map(lambda x: x.replace('\\n', ' '), df.description.tolist()))
df.style.hide_index()

title,description,city,url
"Next to Stadiums! Updated, Modern 1 BD apt","Cozy and comfortable modern 1 BD apt. This apt is in the middle of EVERYTHING. Tons of restaurants and tourist attractions with in 1-2 miles. A second floor unit overlooking the pool. Comfortable king size Serta memory foam mattress in the bedroom, and a comfortable couch and love seat in the living room, along with a queen size air mattress. High speed internet and Netflix provided. This apartment is the perfect place if you are coming to town for the theme Parks or anything at ATT stadium.",Arlington,https://www.airbnb.com/rooms/18363018?location=Bedford%2C%20TX
Jay's Lounge,"hola, Enjoy this very quiet and comfortable room with Smart TV with Netflix, fridge, air conditioner with queen size bed and laundry machine. This converted garage room is centrally located between Dallas and fort worth metroplex. 9.7 miles from DFW International airport. 7 miles from Love Field airport. 11 miles from Dallas downtown area. 6 miles from a mall, AMC, gym and some of the Great Tex-Mex restaurants. Centrally located and very quite and decent neighborhood.",Irving,https://www.airbnb.com/rooms/18655719?location=Coppell%2C%20TX
Jay's Lounge,"hola, Enjoy this very quiet and comfortable room with Smart TV with Netflix, fridge, air conditioner with queen size bed and laundry machine. This converted garage room is centrally located between Dallas and fort worth metroplex. 9.7 miles from DFW International airport. 7 miles from Love Field airport. 11 miles from Dallas downtown area. 6 miles from a mall, AMC, gym and some of the Great Tex-Mex restaurants. Centrally located and very quite and decent neighborhood.",Irving,https://www.airbnb.com/rooms/18655719?location=Colleyville%2C%20TX
Jay's Lounge,"hola, Enjoy this very quiet and comfortable room with Smart TV with Netflix, fridge, air conditioner with queen size bed and laundry machine. This converted garage room is centrally located between Dallas and fort worth metroplex. 9.7 miles from DFW International airport. 7 miles from Love Field airport. 11 miles from Dallas downtown area. 6 miles from a mall, AMC, gym and some of the Great Tex-Mex restaurants. Centrally located and very quite and decent neighborhood.",Irving,https://www.airbnb.com/rooms/18655719?location=Carrollton%2C%20TX
"Next to Stadiums! Updated, Modern 1 BD apt","Cozy and comfortable modern 1 BD apt. This apt is in the middle of EVERYTHING. Tons of restaurants and tourist attractions with in 1-2 miles. A second floor unit overlooking the pool. Comfortable king size Serta memory foam mattress in the bedroom, and a comfortable couch and love seat in the living room, along with a queen size air mattress. High speed internet and Netflix provided. This apartment is the perfect place if you are coming to town for the theme Parks or anything at ATT stadium.",Arlington,https://www.airbnb.com/rooms/18363018?location=Cedar%20Hill%2C%20TX
Spacious South Austin home,"Spacious shaded yard/deck for playing, lounging, grilling (and professionally treated for mosquitoes!) Open kitchen perfect for entertaining- close to Central Market, Whole Foods, numerous restaurants- comfortably sleeps 6-8. Flatscreen TVs in 4 rooms w cable/netflix. Great neighborhood- Kid friendly and perfect for families! One king bed, one queen, one twin, one queen futon, one twin floor mattress if needed",Austin,https://www.airbnb.com/rooms/9970080?location=Buda%2C%20TX
"Next to Stadiums! Updated, Modern 1 BD apt","Cozy and comfortable modern 1 BD apt. This apt is in the middle of EVERYTHING. Tons of restaurants and tourist attractions with in 1-2 miles. A second floor unit overlooking the pool. Comfortable king size Serta memory foam mattress in the bedroom, and a comfortable couch and love seat in the living room, along with a queen size air mattress. High speed internet and Netflix provided. This apartment is the perfect place if you are coming to town for the theme Parks or anything at ATT stadium.",

### 3.2) Conjunctive query & Ranking score
Now the task is to build up a Search Engine that takes a query and returns *top-k* documents related to the query. 

In particular we want:

* Find all the documents that contains all the words in the query (this process is the same that the previous Search Engine do).
* Sort them by their similarity with the query
* Return in output *k* documents, or all the documents with non-zero similarity with the query when the results are less than _k_.

To solve this task, we use the *tfIdf* score, and the _Cosine similarity_. Let's see how.

The Cosine Similarity is a measure of similarity that is useful when two documents are compared. Its main advantage is that's capable of normalize vector's length during comparison.
Another important feature of Cosine Similarity is that is optimal for sparse vectors since we only need to consdier the non-zero dimensions.

The general formula to calculate it it's given by $cos(\theta)=\frac{A \cdot B}{\parallel A \parallel \parallel B \parallel}= \frac{\sum_{i=1}^{n}{A_i \cdot B_i}}{\sqrt{\sum_{i=1}^{n}A_i^2}  \sqrt{\sum_{i=1}^{n}B_i^2} }$

In particular we want to find the similarity between the query and each document:
$cos(\theta)=\frac{Query \cdot doc_i}{\parallel Query \parallel \parallel doc_i \parallel}$

To perform this formula, we want to calculate `TF` and `IDF` to convert text into numbers so that it can be represented by a vector.

With `TF` we mean Term Frequency, or times that a certain term occurred in a query or in a doc. We also want to normalize the document based on its size and we can do it dividing the term frequency by the total number of terms. 

With `IDF` we mean Inverse Document Frequency. Inverse Document Frequencies help us to _weigh down_ the effects of too frequently occurring term(and _weigh up_ less frequent terms). 

We can calculate it using logarithms: $\log (\frac{n}{m})$ with n=Total number of docs and m=Number Of Documents with term game in it.




First thing; we create a new inverted index that contains `tfIdf`s:

In [87]:
# from the vocabulary the inverted index is created. In this case the index are mapped to a tuple (doc, tf-idf)

inverted_index_2 = {}

for file in range(18259):

    op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\doc\doc_' + str(file) + '.tsv', 'r', encoding="utf-8")
    for line in op:
        ou = line.strip().split('\t')
        sentence = ou[5].replace('\\n', ' ').replace('/', ' ').replace('*', ' ').replace('\\r', ' ').replace('\\t', ' ') + ' ' + ou[8].replace('\\n', ' ').replace('/', ' ').replace('*', ' ').replace('\\r', ' ').replace('\\t', ' ')
    op.close()
 
    
    # preprocessing data deleting stop words, punctuations, ecc.  
    final = preprocess(word_tokenize(sentence))
    
    
    # CREATING INVERTED INDEX with tf-idf values
    for word in final:
        index = vocabulary[word]
        
        tf = final.count(word) / len(final)
        idf = math.log( 18259 / len(inverted_index[vocabulary[word]]))
        tf_idf = tf*idf
        
        if not (index in inverted_index_2):
            inverted_index_2[index] = [('doc_' + str(file), tf_idf )]
        elif not(('doc_' + str(file), tf_idf)  in inverted_index_2[index]):
            inverted_index_2[index] = inverted_index_2[index] + [('doc_' + str(file), tf_idf)]


op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\inverted_index_2.txt', 'w', encoding="utf-8")
op.write(json.dumps(inverted_index_2))
op.close()

In [20]:
# opening the created inverted_index_2.txt

op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\inverted_index_2.txt', 'r', encoding="utf-8")
inverted_index_2 = json.loads(op.read())


Now we can handle query input:

In [40]:
user_query = input('What are you searching?\n')

# preprocessing data deleting stop words, punctuations, ecc.  
query = preprocess(word_tokenize(user_query))


What are you searching?
a queen and comfortable room with netflix


In [41]:
# taking from the inverted index only the words present in the query
voc = {}
inverted_index_query = {}
i=0
for word in query: 
    voc[i]  = vocabulary[word]
    i = i+1
for index in range(i):
    inverted_index_query[voc[index]] = inverted_index[voc[index]]

# finding list of docs that contains all the words in the query
docs = []

for i in range(18259):
    doc = 'doc_' + str(i)
    b = True
    for j in voc.values(): 
        b = b and (doc in inverted_index_query[j])
    if b:
        docs.append(i)
docs_and = set(docs)


Traducing the query in our 'language':

In [42]:
voc = {}
inverted_query = {}
i=0

# taking only the words in query from the inverted_index_2
for word in query: 
    voc[i]  = vocabulary[word]
    i = i+1
for index in range(i):
    inverted_query[voc[index]] = inverted_index_2[voc[index]]



We normalize the document vector :

In [43]:
n = {}
index = 0

# normalization of the document vector to the length of the query  
for i in inverted_query:
    for j in inverted_query[i]:
        if index == 0:
            n[j[0]] = [j[1]]
        elif (not (j[0] in n)):
            n[j[0]] = [0]*index + ([j[1]])
        else:   
            n[j[0]] = n[j[0]] + [0]*(index - len(n[j[0]])) + ([j[1]])
    index = index + 1

    
for i in n:
    if len(n[i]) < len(query) :
        n[i] = n[i] + [0]*(len(query)-len(n[i]))


Finding `tfidf`s of the query:

In [44]:
# calculating tf-idf of the query
tfidf_query = []  
for word in query:
    tf_query = query.count(word) / len(query)
    idf_query = math.log( 18259 / len(inverted_index[vocabulary[word]]))
    tfidf_query.append(tf_query * idf_query)


Finding denominator of our dear formula: norm of the query and norm of docs:

In [45]:
# norm of the query
norm_query = numpy.sqrt(sum(list(map(lambda x: x**2, tfidf_query))))

In [46]:
# norm of each 'normalized' document 
norm_doc = {}
for i in range(18259):
    doc = 'doc_' + str(i)
    nomin = 0
    for i in inverted_query:
        for j in inverted_query[i]:
            if j[0] == doc :
                nomin = nomin + j[1]**2
                norm_doc[doc] = nomin
for i in norm_doc:
    norm_doc[i] = numpy.sqrt(norm_doc[i])


Finding finally our cosine similarities, find the set union with documents that contains **all** the words of the query, and sorting them with haep algorithms:

In [47]:
# finding the cosine similarities for each word
cosines= []
for i in n:
    st = i
    cos = numpy.dot(tfidf_query, n[st])
    cosine = round(cos / (norm_doc[st]*norm_query), 10)

    cosines.append((cosine, i ))


In [48]:
cosines_2 = [] 
for i in cosines:
    if int(i[1][4:]) in docs_and:
        cosines_2.append(i)


In [49]:
# create the max heap from the cosine similarituìies values and taking the k-largest values
k = 6

heap_max = []
for i in cosines_2:
    heapq_max.heappush_max(heap_max, i)

best_cosine = []
docs = []
for i in range(k):
    try:
        cos = heapq_max.heappop_max(heap_max)
        best_cosine.append(cos[0])
        docs.append(int(cos[1][4:]))
    except:
        break

Printing them in the format that we want:

In [50]:
# print the table with the best k documents found and their ranking

df = f.filter(items = ['title', 'description', 'city', 'url']).loc[docs]
df.description = list(map(lambda x: x.replace('\\n', ' '), df.description.tolist()))
df['ranking'] = best_cosine
df.style.hide_index()

title,description,city,url,ranking
Jay's Lounge,"hola, Enjoy this very quiet and comfortable room with Smart TV with Netflix, fridge, air conditioner with queen size bed and laundry machine. This converted garage room is centrally located between Dallas and fort worth metroplex. 9.7 miles from DFW International airport. 7 miles from Love Field airport. 11 miles from Dallas downtown area. 6 miles from a mall, AMC, gym and some of the Great Tex-Mex restaurants. Centrally located and very quite and decent neighborhood.",Irving,https://www.airbnb.com/rooms/18655719?location=Carrollton%2C%20TX,0.978778
Jay's Lounge,"hola, Enjoy this very quiet and comfortable room with Smart TV with Netflix, fridge, air conditioner with queen size bed and laundry machine. This converted garage room is centrally located between Dallas and fort worth metroplex. 9.7 miles from DFW International airport. 7 miles from Love Field airport. 11 miles from Dallas downtown area. 6 miles from a mall, AMC, gym and some of the Great Tex-Mex restaurants. Centrally located and very quite and decent neighborhood.",Irving,https://www.airbnb.com/rooms/18655719?location=Colleyville%2C%20TX,0.978778
Jay's Lounge,"hola, Enjoy this very quiet and comfortable room with Smart TV with Netflix, fridge, air conditioner with queen size bed and laundry machine. This converted garage room is centrally located between Dallas and fort worth metroplex. 9.7 miles from DFW International airport. 7 miles from Love Field airport. 11 miles from Dallas downtown area. 6 miles from a mall, AMC, gym and some of the Great Tex-Mex restaurants. Centrally located and very quite and decent neighborhood.",Irving,https://www.airbnb.com/rooms/18655719?location=Coppell%2C%20TX,0.978778
Spacious South Austin home,"Spacious shaded yard/deck for playing, lounging, grilling (and professionally treated for mosquitoes!) Open kitchen perfect for entertaining- close to Central Market, Whole Foods, numerous restaurants- comfortably sleeps 6-8. Flatscreen TVs in 4 rooms w cable/netflix. Great neighborhood- Kid friendly and perfect for families! One king bed, one queen, one twin, one queen futon, one twin floor mattress if needed",Austin,https://www.airbnb.com/rooms/9970080?location=Buda%2C%20TX,0.952353
"Next to Stadiums! Updated, Modern 1 BD apt","Cozy and comfortable modern 1 BD apt. This apt is in the middle of EVERYTHING. Tons of restaurants and tourist attractions with in 1-2 miles. A second floor unit overlooking the pool. Comfortable king size Serta memory foam mattress in the bedroom, and a comfortable couch and love seat in the living room, along with a queen size air mattress. High speed internet and Netflix provided. This apartment is the perfect place if you are coming to town for the theme Parks or anything at ATT stadium.",Arlington,https://www.airbnb.com/rooms/18363018?location=Cedar%20Hill%2C%20TX,0.881165
"Next to Stadiums! Updated, Modern 1 BD apt","Cozy and comfortable modern 1 BD apt. This apt is in the middle of EVERYTHING. Tons of restaurants and tourist attractions with in 1-2 miles. A second floor unit overlooking the pool. Comfortable king size Serta memory foam mattress in the bedroom, and a comfortable couch and love seat in the living room, along with a queen size air mattress. High speed internet and Netflix provided. This apartment is the perfect place if you are coming to town for the theme Parks or anything at ATT stadium.",Arlington,https://www.airbnb.com/rooms/18363018?location=Bedford%2C%20TX,0.881165


## Step 4: Define a new score!
Now we suppose that Airbnb wants us in their team, and they need the best idea ever on how to rank the houses based on the queries of their users.

In this scenario, a single user can give in input more information than the single textual query, and we want to take into account all this informations, and think a creative and logical way on how to answer at user's requests.

We decided to ask user other infos and then to write a new scoring function.

### ADDING NEW VARIABLES
We will add some new variables such as amenities that the host can provide and can not provide, whether the host is superhost or not, number of guest(s), number of bath(s), number of bed(s) and average reviews ranking stars. All those information are retrieved from the url of each Airbnb property.

Retrieving all the informations directly from the Airbnb site takes a lot of time. In order to avoid to do this work each time, we saved all the data in the file `airbnb_newdata.csv` that is present in the GitHub repository. 

All the informations were taken from Airbnb thanks to the library `Beautiful Soup`. Some rooms do not exist anymore or are missing some data, in those case the rooms are filles with Nan.

In [11]:
# Creating data by saving simultaneously

# This process will do just one time since it takes some time. We upload the file on GitHub, you can reach there.

path = r'D:\\Data Science\\ADM(Algorithmic Methods of Data Mining)\\hw\\hw3\\hw3 data\\'

file_index = []
with open(path + 'airbnb_newdata.csv', encoding='utf-8') as csvfile:
    readCSV = csv.reader(csvfile)
    
    for row in readCSV:
        file_index.append(row[1])
file_index = list(map(str, file_index))        
csvfile.close()
    

for row in f.values:
    try:
        check_file_index = str(row[0])
        
        if check_file_index not in set(file_index):
            url = str(row[9])
            content = requests.get(url)
            soup = BeautifulSoup(content.text, "lxml")

            newrow = list(row)

            newrow.append(listing_amenities(content, soup)[0])
            newrow.append(listing_amenities(content, soup)[1])
            newrow.append(is_superhost(content, soup))
            newrow.append(guest_no(content, soup))
            newrow.append(bath_type(content, soup))
            newrow.append(bed_no(content, soup))
            newrow.append(rating_value_reviews(content, soup))

            newrow = pd.DataFrame([newrow]) 

            newrow.to_csv(path + 'airbnb_newdata.csv', mode='a', header=False, encoding='utf-8')
    
    except:
        pass # at some point Airbnb can block to you to enter the url that we're trying to get some more information.

In [ ]:
# opening new data that we created with new variables

# remember that some links are not still alive, in that case we got all NaN.

path = r'D:\\Data Science\\ADM(Algorithmic Methods of Data Mining)\\hw\\hw3\\hw3 data\\'
f = pd.read_csv(path + 'airbnb_newdata.csv', index_col=None)

In [ ]:
# Create a list for all amenities 

all_amenities = []
for i in range(len(f.iloc[:,11])):
    row = f.iloc[:,11][i]
    if not(pd.isnull(row)):
        row_list = row.split(",")
        for j in range(len(row_list)):
            if row_list[j] not in all_amenities:
                all_amenities.append(row_list[j])


In [14]:
# Create check button so that the guest can choose features for place
from tkinter import *
def pick_features():
    Window = Tk()
    Window.config(bg='white')
    var = []
    choices = []

    def save_choices():
        for v in var:
            choices.append(False) if v.get() =="off" else choices.append(True)
    
    Label(Window, text="Please, choose your amenities preference",fg='black',bg='white',font=("Arial Bold", 15)).grid(row=0,column=1)
    
    row = 1   
    col = 0
    for i in range(len(all_amenities)):
        var.append(StringVar())
        for v in var:
            v.set("off")
        Checkbutton(Window, text=all_amenities[i], variable=var[-1],onvalue='on', offvalue='off',
                   activebackground="green",bg='white',fg='black').grid(row=row,column=col)
        #cb.pack()

        row += 1
        if row>math.ceil(len(all_amenities)/4):
            col += 1; row = 1

    Button(Window, text="Please, send your preference", command=save_choices,
           font=("Arial Bold", 10),fg='green',bg='white').grid(row=row+1,column=col)
    Button(Window, text="QUIT", command=Window.destroy, font=("Arial Bold", 10),
           fg='red',bg='white').grid(row=row+2,column=col)

    Window.mainloop()

    choosen_index = numpy.where(choices)[0]
    choosen = list(pd.Series(all_amenities)[choosen_index])
    return choosen

### USER QUERIES

After the inizialization part, it is time to start asking the user what are his preferences. 

We ask the customer 5 questions:
1. Where do you want to go? 
2. How much do you want to spend?
3. How many bedrooms do you want?
4. How many guest will stay?
5. Which place's amenities do you want?

Once the user answers these questions, we start to give a rank to each rooms. How the ranking is assigned is described later.

In [15]:
# asking the user where he/she wanna go
user_city = input('Where do you want to go?\n')

# asking the user how much he want to spend
user_price = input('How much do you want to spend?\n')

# asking the user how many bedrooms the client wants
user_bedroom = input('How many bedrooms do you want?\n')

# asking the user how many guest will stay
user_guest = input('How many guest will stay?\n')

# asking the user which place's amenities the client wants
print("Which place's amenities do you want?\n")
user_amenities = pick_features()
print(user_amenities)

Where do you want to go?
dallas
How much do you want to spend?
150
How many bedrooms do you want?
2
How many guest will stay?
4
Which place's amenities do you want?

['Kitchen', 'Wifi', 'Air conditioning', 'Breakfast']


###### Check Button
The number of features that an apartment/room could have is really high. To make the user more comfortable with the choice, we have created an interactive windows where the customer is able to select his preferred features.

We will take user choice(s) for their amenities peference by using check button, which is figured below.

![](CheckButton.PNG?raw=true)

#### CITY RANKING

The most important decision that the custumer has to make is where to go. In this part we give a score in the range [0, 1].

A score close to one means that the distance between the analyzed room and the location given by the user is closed to zero. More than a room is far from the user choice, more that room city rank goes to zero.

In order to give a score we decided to use the cumulative distribution function, in this form:

$ city Rank = 1 - \frac{x - minimum}{maximum - minimum} $

where x is the location of the room being analyzed, minimum is the closest room to the user location and maximum is the farest room from the user loc.

In [16]:
# taking from the dataframe only the cities with their means of coordinates
f['city'] = list(map(lambda x: x.lower(), f['city']))
f.at[1216, 'latitude'] = 31.498774
f.at[1216, 'longitude'] = -94.345574
geo_city = f.groupby('city').mean().filter(['latitude', 'longitude'])

In [17]:
# find the coordinates for the inserted city
coord_city = (geo_city[geo_city.index == user_city]['latitude'].values[0], geo_city[geo_city.index == user_city]['longitude'].values[0])
print('Coordinates for the request city: ', coord_city)

Coordinates for the request city:  (32.84701372104991, -96.78911736186348)


In [18]:
# fill the NaN latitudine and longitude with the mean location of the city 
for i in f[numpy.isnan(f.latitude)].index:
    latitude = (geo_city.loc[(f.loc[i, 'city'])][0])
    longitude = (geo_city.loc[(f.loc[i, 'city'])][1])

    f.at[i, 'latitude'] = latitude
    f.at[i, 'longitude'] = longitude

In [19]:
# distance betweeen user coords and room coords 
dist = []

index = 0
for i in f.index:
    dist.append(distance.distance((f.loc[i, 'latitude'], f.loc[i, 'longitude']), coord_city).km)  


In [20]:
# rankings for city
rankings_city = {}
for i in range(len(dist)):
    if(dist[i] >= 0):
        rankings_city[i] = 1 - ((dist[i] - min(dist) ) / (max(dist)- min(dist)))
    else:
        rankings_city[i] = None
rankings_city    
f['city_rank'] = rankings_city.values()

In [21]:
rankings_city = {}
rang  = max(dist) - min(dist) 

# rank to each city in Texas. Highest ranks are for the closest cities to the user's one. 
for city in geo_city.index:
    coord = (geo_city[geo_city.index == city]['latitude'].values[0], geo_city[geo_city.index == city]['longitude'].values[0])
    rank = 1 - ((distance.distance(coord, coord_city).km - min(dist)) / rang)
    rankings_city[city] = rank

#### PRICE RANKING
Another important decision of the user is how much he want to spend. Since the prices of the rooms and apartments vary a lot, we have decided to give a score, in [0,1], close to 1 if the room price is close to the customer pick, or close to zero otherwise.

The range between the cheapest and the more expensive rooms is really high, so it does not make sense to use the uniform distribution as in the previous section. In this part we found that the logarithm was a good choice to normalized the rank.

The formula used was:

$priceRank = 1 - \frac {log (priceDistance + 1)}{10}$

where priceDistance is the distance between the user decision and the analyzed room.

In [22]:
# column of average price without the '$' char
f['price_rank'] = list(map(lambda x: str(x)[1:], f['average_rate_per_night']))

In [23]:
# distances between the average price of the room and the user price
price_dist=[]
for i in f.price_rank.values:
    try:
        price_dist.append(abs((int(i)) - int(user_price ))  )
    except:
        price_dist.append(-1)

In [24]:
# taking the range (min and max) from the distances
minimum = price_dist[0]
for i in price_dist:
        if i < minimum and not(i == -1):
            minimum = i
rang = max(price_dist) - minimum           

In [25]:
# finding the rankings
# logarithm cause the range min-max is really high
rankings_price = {}
for i in range(len(price_dist)):
    if(price_dist[i] >= 0):
        rankings_price[i] = 1-(numpy.log(price_dist[i]+1))/10
    else:
        rankings_price[i] = None
rankings_price    
f['price_rank'] = rankings_price.values()

#### BEDROOM RANKING

Third decision of the user is the number of rooms. The score is always a number in [0,1] assigned in the same way as described above. 

It was possible to use the uniform distribution as in the city rank, as shown here:

$ bedroom Rank = 1 - \frac{x - minimum}{maximum - minimum} $


In [26]:
# taking the distances between the #bedrooms of the room and the user number
room_dist=[]
for i in f.bedrooms_count.values:
    try:
        if i == 'Studio':
            room_dist.append(abs(1.5 - int(user_bedroom )))
        else:
            room_dist.append(abs( (int(i)) - int(user_bedroom)))
    except:
        room_dist.append(-1)

In [27]:
# taking the range (min and max) from the distances
minimum = room_dist[0]
for i in room_dist:
        if i < minimum and not(i == -1):
            minimum = i
rang = max(room_dist) - minimum           

In [28]:
# finding the rankings for bedrooms
rankings_bedroom = {}
for i in range(len(room_dist)):
    if(room_dist[i] >= 0):
        rankings_bedroom[i] = 1 - ((room_dist[i] - minimum ) / rang)
    else:
        rankings_bedroom[i] = None
rankings_bedroom    
f['bedroom_rank'] = rankings_bedroom.values()

#### GUEST RANKING
Forth decision of the user is the number of guest that will be present. Same as before a rank in [0,1] is given, always with the uniform distribution that follows.

$ guest Rank = 1 - \frac{x - minimum}{maximum - minimum} $


In [29]:
# taking the distances between the #guest who can stay in the host's plae and the user number
guest_dist = []
for i in f.guest.values:
    try:
        guest_dist.append(abs((int(i)) - int(user_guest)))
    except:
        guest_dist.append(-1)

In [30]:
# taking the range (min and max) from the distances
minimum = guest_dist[0]
for i in guest_dist:
        if i < minimum and not(i == -1):
            minimum = i
range_guest = max(guest_dist) - minimum   

In [31]:
# finding the rankings for guest number
rankings_guest = {}
for i in range(len(guest_dist)):
    if(guest_dist[i] >= 0):
        rankings_guest[i] = 1 - ((guest_dist[i] - minimum ) / range_guest)
    else:
        rankings_guest[i] = None
rankings_guest    
f['guest_rank'] = rankings_guest.values()

#### REVIEWS STAR RANKING
Since the guest can vote the host from 0 to 5, it is already scaled. We will just standardized it.

In [130]:
stars = [None if pd.isnull(s) else int(s)/5 for s in f.stars]
f['stars_rank'] = stars

#### AMENITIES RANKING

The amenities rank is related to the numbers of the features that a room/apartment gives according to the user choises. More amenities the room have, higher is its rank. 

In [32]:
# finding the rankings for amenities
ranking_amenities = []
for i in f.amenities.values:
    try:
        count = 0
        for j in user_amenities:
            if j in i:
                count += 1
        ranking_amenities.append(round(count/len(user_amenities),2))
    except:
        ranking_amenities.append(None)
ranking_amenities
f['amenities_rank'] = ranking_amenities

#### SUPERHOST RANKING
With the info taken from the links we were able to know if the host is a Superhost or not. 

#### FINAL RANKING
Calculated all the different rankings, it is time to assign a final rank to each room. We decided to give different weights to all the ranks according to what we consider more important. 

These are the weights (in percentage):

- location: 70%
- price: 10%
- number of guest: 7%
- number of amenties: 4%
- number of bedrooms: 4%
- number of stars: 3%
- superhost: 2%

If one of the rankings is missing, its weight is redistributed through the others.

In [136]:
# giving weithed priority to the different rankings

# weights
w1 = 0.10 #for price 
w2 = 0.02 #for superhost
w3 = 0.03 #for reviews stars
w4 = 0.70 #for city ranking
w5 = 0.04 #for bedroom 
w6 = 0.07 #for guest 
w7 = 0.04 #for amenities 
w = [w1, w2, w3, w4, w5, w6, w7]

# our criterias
cri =  ['price_rank','superhost','city_rank','bedroom_rank','guest_rank','stars_rank','amenities_rank']

rank = []

for i in f.index:
    nan_idx = []
    func_idx = []
    for j in range(len(f.loc[i,cri])):
        if (str(f.loc[i,cri][j])=='nan') or (f.loc[i,cri][j] is None):
            nan_idx.append(j) # if we have nan or none, add nan indexes
        else:
            func_idx.append(j) # otherwise, add them to use in function
            
    add_up = sum(pd.Series(w)[nan_idx])/(len(w)-len(nan_idx)) # find the averange weight to add up the weight that will be in our formula
    updated_weights = list(pd.Series(w)[func_idx]+add_up) # split the weights which come from nan/none variables
    updated_cri = list(pd.Series(cri)[func_idx]) # criterias which we will use in our formula
    
    r = 0
    for c in range(len(updated_weights)):
        if updated_cri[c] == 'superhost':
            if f.loc[i,updated_cri[c]] == 'T':
                superhost_rank = 1
            else:
                superhost_rank = 0
                
            r = r + superhost_rank*updated_weights[c]
        else:
            r = r + f.loc[i,updated_cri[c]]*updated_weights[c] # formula = criteria1*w1 + criteria2*w2 + ...
    
    # rank.append(str(round(r*100, 3)) + '%')
    rank.append(round(r*100, 3))
    
f['final_rank'] = rank       

In [166]:
# create the max heap from the cosine similarities values and taking the k-largest values

# fr includes all final rank for each doc
fr = []
for i in f.index:
    fr.append(float(f.loc[i , 'final_rank']))
    
k = 10 # how many options we want to show
top_k = heapq.nlargest(k, enumerate(f.final_rank), key=lambda x: x[1])

indexes = [e[0] for e in top_k]

# displaying the best k rooms 

df = f.loc[indexes].filter(items = ['title', 'description', 'city', 'url'])
df.style.hide_index()

title,description,city,url
Tudor Home Guest House,"Charming 2 bedroom guest house. Large kitchen/dining. 1 large bath. Private garden entrance. Walk to the Arboretum, White Rock Lake, shops, bars, dining, etc. $150 per night up to 4 ppl. $25 per night for each additional. $50 cleaning fee .",dallas,https://www.airbnb.com/rooms/6324590?location=Balch%20Springs%2C%20TX
Eclectic Industrial Loft in Deep Ellum.,"The loft is in a prime location with true walkability to all things Deep Ellum while offering the conveniences of a home: stainless steel appliances, updated bathroom, washer/dryer, wifi, gated/covered parking, open kitchen/den floor plan and split bedrooms – one up, one down. You will get the full historic experience of Deep Ellum with the industrial finish out which includes, soaring 20 ft ceilings, exposed brick & ductwork, and concrete floors. Access to common rooftop for skyline views.",dallas,https://www.airbnb.com/rooms/18895401?location=Balch%20Springs%2C%20TX
Eclectic Industrial Loft in Deep Ellum.,"The loft is in a prime location with true walkability to all things Deep Ellum while offering the conveniences of a home: stainless steel appliances, updated bathroom, washer/dryer, wifi, gated/covered parking, open kitchen/den floor plan and split bedrooms – one up, one down. You will get the full historic experience of Deep Ellum with the industrial finish out which includes, soaring 20 ft ceilings, exposed brick & ductwork, and concrete floors. Access to common rooftop for skyline views.",dallas,https://www.airbnb.com/rooms/18895401?location=Arlington%2C%20TX
Eclectic Industrial Loft in Deep Ellum.,"The loft is in a prime location with true walkability to all things Deep Ellum while offering the conveniences of a home: stainless steel appliances, updated bathroom, washer/dryer, wifi, gated/covered parking, open kitchen/den floor plan and split bedrooms – one up, one down. You will get the full historic experience of Deep Ellum with the industrial finish out which includes, soaring 20 ft ceilings, exposed brick & ductwork, and concrete floors. Access to common rooftop for skyline views.",dallas,https://www.airbnb.com/rooms/18895401?location=Carrollton%2C%20TX
Cottage charmer near TCU,"One entire side of the duplex is available. Combines some style of the past with the convenience of the future. Charming cottage feel. Terrific for TCU Football, Baseball, Parent's Weekend and Graduation. Perfect for Fort Worth visitors- easy access to the Zoo, Will Roger's Coliseum, Downtown, Cultural District, shopping, dining and entertainment.\nVery comfortable happy home! Welcome!!",fort worth,https://www.airbnb.com/rooms/18912605?location=Burleson%2C%20TX
Cottage charmer near TCU,"One entire side of the duplex is available. Combines some style of the past with the convenience of the future. Charming cottage feel. Terrific for TCU Football, Baseball, Parent's Weekend and Graduation. Perfect for Fort Worth visitors- easy access to the Zoo, Will Roger's Coliseum, Downtown, Cultural District, shopping, dining and entertainment.\nVery comfortable happy home! Welcome!!",fort worth,https://www.airbnb.com/rooms/18912605?location=Cleburne%2C%20TX
Cottage charmer near TCU,"One entire side of the duplex is available. Combines some style of the past with the convenience of the future. Charming cottage feel. Terrific for TCU Football, Baseball, Parent's Weekend and Graduation. Perfect for Fort Worth visitors- easy access to the Zoo, Will Roger's Coliseum, Downtown, Cultural District, shopping, dining and entertainment.\nVery comfortable happy home! Welcome!!",fort worth,https://www.airbnb.com/rooms/18912605?location=Benbrook%2C%20TX
Uptown House,the apartment is a great place to be,mckinney,https://www.airbnb.com/rooms/18960232?location=Carrollton%2C%20TX
Uptown House,the apartment is a great place to be,mckinney,https://www.airbnb.com/rooms/18960232?location=Anna%2C%20TX
Uptown House,the apartment is a great place to be,mckinney,https://www.ai

## Bonus Step: Make a nice visualization!


An important feature of Airbnb, it's the search on the map. Here we want to try to recreate a similar tool!

Our tool will works in the following way:

1. Takes in input a set of coordinates, e.g: [32.779594, -96.800768], and a maximum distance from the coordinates, e.g.: 10km.
2. Generate a map, with a circle of the given radius, where the center is represented by the coordinates given in input.
3. Shows the houses that are inside the circle of the given radius, coloring their marker according to the average price per night.

The first thing that we want to do is to create our dataframe.


In [4]:
f2 = f.filter(items = ['Unnamed: 0','latitude','longitude', 'title','average_rate_per_night','url'])

Now we let the user enter coordinates and radius that he wants:

In [5]:
latitude=float(input('Enter latitude:'))
longitude=float(input('Enter longitude:'))
radius=float(input('Enter radius in km:'))
coord=[latitude, longitude]

Enter latitude:32.779594
Enter longitude:-96.800768
Enter radius in km:10


In [6]:
houses=[]
for i in range(18259):
    try:
        coord_houses=[f2['latitude'][i],f2['longitude'][i]]
        distnz=distance.distance(coord, coord_houses).km
        if distnz<radius:
            houses.append(i)
    except: pass

In [7]:
f3 = f2.loc[houses]

Now we want to create the map. To achive that, we perform this steps:

    1. Define a function that help us to visualize the price of the house associating it with the color of the marker.
    2. Create a map centered in the point given by the user.
    3. Add to the map a circle with given radius centered in the same point.
    4. Add a marker for every house that we found.
    5. Add the legend to our map.
    6. Display the map.

In [8]:
#1 : Define the function of markers colors
def colorfunction():
    price= int(f3['average_rate_per_night'][house][1:])
    if price > 500: 
        col='black'
    if price < 500: 
        col='red'
    if price < 100: 
        col='orange'
    if price < 50: 
        col='green'
    return col

In [9]:
legenda= cm.StepColormap(['green','orange','red','black'], index=[0, 50, 100, 500], vmin=0, vmax=750)

In [10]:
#2. Create the map
Map=folium.Map(
    location=coord,
    zoom_start=11, 
    tiles="cartoDBpositron"
)

#3. Add the circle with radius 10 km:
folium.Marker(
    location=coord,
    popup='Location',
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(Map)
folium.Circle( 
    location=coord,
    radius=radius*1000,
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(Map)

#4. Add a marker for every house
for house in f3.T: 
    house_coord=[f3['latitude'][house],f3['longitude'][house]] #house coord
    folium.Marker(
    location=house_coord,
    popup=folium.Popup('<a href=' + f3['url'][house] + '>'+f3['average_rate_per_night'][house]+' </a>'), #airbnb page of house
    icon=folium.Icon(color=colorfunction(), icon='home')
).add_to(Map)
    
#5. Add the legend
colormap = legenda
colormap.caption = 'Average price per night'
Map.add_child(colormap)

Map.save('ap.html')
#6. Display the map
Map

If you can't see the map probably it's becouse you didn't read the readme, so you are not inside the nbviewer version of our notebook(read carefully!).

Anyway, this is the NBVIEWER to see the html of the map in full page.
http://nbviewer.jupyter.org/github/aleflabo/ADM-HW_10/blob/master/ap.html